In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
os.path.abspath('')

'/mnt/e/PreliminaryAnalysisGrandCanyon/src/inferenceLSTM/newML'

In [4]:
win = 15
fileNom = "trajFile_mvWin"
#warnings.filterwarnings("ignore")
cwd = os.path.abspath('')
subjectList = pd.read_csv(cwd+'/participantID1.csv', header=None) # import the number participant IDs without the header
numSubjects = subjectList.size
subjectList = subjectList.map(str) # convert to string
subjectList = subjectList.values.tolist() # convert to python list
subjectAlias = np.linspace(1,numSubjects,numSubjects).tolist()
trialNames = ['NN','YN','NY','YY'] # NN = NNL + NNH, YN = YNL + YNH, NY = NYL + NHH, YY = YYL + YYH
trialNum = [['111','112' ],['211','212'],['121','122'],['221','222']] # do not delete this line

c = 1
d = 1
trajectory = [] # variable that will store input data in [number of samples, length of seach segment, features] format
output = [] # variable that will store true output data as described above
t = 0
correspondingSubject = [-1,-1]

for subjects, subNum in zip(subjectList, subjectAlias):
    tempOutputAggregator = []
    for trials, index in zip(trialNum,range(len(trialNum))):
        tempOutputAggregator = []
        for trial in trials:
            lookFolder = cwd + '/exportedTrajectory_'+str(win)+'/'+subjects[0]+'/'+ trial # path to folder where traj.csv is stored
            tempTraj = pd.read_csv(lookFolder+'/'+ fileNom +'.csv', header=None) # read traj.csv file
            segmentSize = pd.read_csv(lookFolder+'/windowSize.csv', header=None) # read segmentSize.csv file
            tempTraj = tempTraj.to_numpy() # convert pandas dataframe to numpy array
            segmentSize = segmentSize.to_numpy() # convert pandas dataframe to numpy array
            arrShapeSup = tempTraj.shape # get shape of csv file1
            numSegments = arrShapeSup[0]/segmentSize # number of trial segments in traj.csv file
            tempTrajAggregator = [] # reset tempTrajAggregator
            chunkStart = 0 # initial chunk position
            print("Array Length {}, Segment size {} ".format(arrShapeSup[0],segmentSize))
             # loop to separate chunks of trajectory data and format it as [., length of each segment, features] 
            for i in range(int(arrShapeSup[0]- segmentSize[0][0]+1)):
                # obtain a chunk of trajectory defined by segmentSize.csv file
                #print(i)
                tempTrajChunk = tempTraj[i:i+int(segmentSize[0][0]),:] 
                arrShape = tempTrajChunk.shape # get the shape of the chunk
                # reshape the chunk to [1, length of seach segment, features]
                tempTrajChunk = tempTrajChunk.reshape(-1,arrShape[0],int(arrShape[1]))
                chunkStart = chunkStart + int(segmentSize[0][0]) # update chunk position by segment size
                if len(tempTrajAggregator) > 0:
                    # for 1+ run, append tempTrajAggregator with data from tempTrajChunk 
                    tempTrajAggregator = np.append(tempTrajAggregator,tempTrajChunk, axis=0)
                if len(tempTrajAggregator) == 0:
                    # for 1st run, set tempTrajAggregator equal to tempTrajChunk
                    tempTrajAggregator = tempTrajChunk

                if len(correspondingSubject) > 0:
                    # for 1+ run, append tempTrajAggregator with data from tempTrajChunk 
                    correspondingSubject.append(subNum)
                if len(correspondingSubject) == 0:
                    # for 1st run, set correspondingSubject equal to subNum
                    correspondingSubject = subNum

            for i in range(int(arrShapeSup[0]- segmentSize[0][0]+1)):
                tempOutput = 0*np.ones((1,2))
                if index == 0:
                    tempOutput[0][0] = 1e-5
                    tempOutput[0][1] = 1e-5
                if index == 1:
                    tempOutput[0][0] = 1-1e-5
                    tempOutput[0][1] = 1e-5
                if index == 2:
                    tempOutput[0][0] = 1e-5
                    tempOutput[0][1] = 1-1e-5
                if index == 3:
                    tempOutput[0][0] = 1-1e-5
                    tempOutput[0][1] = 1-1e-5
                if len(tempOutputAggregator) > 0:
                    # for 1+ run, append tempOutputAggregator with data from tempOutput 
                    tempOutputAggregator = np.append(tempOutputAggregator,tempOutput, axis=0)
                if len(tempOutputAggregator) == 0:
                    # for 1st run, set tempOutputAggregator equal to tempOutput
                    tempOutputAggregator = tempOutput
            
            if c == 1:
                # for 1st run, set trajectory equal to tempTrajAggregator
                trajectory = tempTrajAggregator
            if c > 1:
                # for 1+ run, append trajectory with data from tempTrajAggregator 
                trajectory = np.append(trajectory,tempTrajAggregator, axis=0)
            c = c + 1
        
        if d == 1:
            # for 1st run, set output equal to tempOutputAggregator
            output = tempOutputAggregator
        if d > 1:
            # for 1+ run, append output with data from tempOutputAggregator 
            output = np.append(output,tempOutputAggregator, axis = 0)
        d = d + 1
correspondingSubject = correspondingSubject[2: len(correspondingSubject)]
arrShape = trajectory.shape
seq_length = arrShape[1] # length of each segment
numSamples = arrShape[0] # number of samples
features = arrShape[2] # data dimensionality



Array Length 12772, Segment size [[360]] 
Array Length 3789, Segment size [[360]] 
Array Length 13237, Segment size [[360]] 
Array Length 13077, Segment size [[360]] 
Array Length 12893, Segment size [[360]] 
Array Length 13014, Segment size [[360]] 
Array Length 1217, Segment size [[360]] 
Array Length 1206, Segment size [[360]] 
Array Length 1016, Segment size [[360]] 
Array Length 2840, Segment size [[360]] 
Array Length 2004, Segment size [[360]] 
Array Length 490, Segment size [[360]] 
Array Length 12694, Segment size [[360]] 
Array Length 1450, Segment size [[360]] 
Array Length 7364, Segment size [[360]] 
Array Length 1604, Segment size [[360]] 
Array Length 11960, Segment size [[360]] 
Array Length 891, Segment size [[360]] 
Array Length 11012, Segment size [[360]] 
Array Length 865, Segment size [[360]] 
Array Length 8228, Segment size [[360]] 
Array Length 8553, Segment size [[360]] 
Array Length 1216, Segment size [[360]] 
Array Length 1632, Segment size [[360]] 
Array Lengt

In [9]:
trajectoryDataFrame = []
for i in range(trajectory.shape[1]):
    #writematrix([xDot',yDot',zDot',subjectTurnRate',subjectHeading,subjectGroundSpeed', subjectSpeed', subjectAccel', subjectZPos, subjectHeightDot'],saveTrajFile);
    df = pd.DataFrame(trajectory[:,i,:], columns = ['speed','thetaDot'], index = correspondingSubject)
    trajectoryDataFrame.append(df)
outputDataFrame =  pd.DataFrame(output, columns = ['Person', 'Terrain'], index = correspondingSubject)

In [10]:
output.shape

(968789, 2)

In [11]:
arrShape[0]

968789

In [12]:
trajectory.shape

(968789, 144, 2)

In [14]:
import pickle
with open('trajectoryDataFrame_win' +str(win)+'_multilabel.pkl', 'wb') as file: 
    pickle.dump(trajectoryDataFrame, file)
with open('outputDataFrame_win'+str(win)+'_multilabel.pkl', 'wb') as file: 
    pickle.dump(outputDataFrame, file)

===============================================================************==================================================================

In [8]:
win = 10
fileNom = "trajFile_nonNorm"
#warnings.filterwarnings("ignore")
cwd = os.path.abspath('')
subjectList = pd.read_csv(cwd+'/participantID1.csv', header=None) # import the number participant IDs without the header
numSubjects = subjectList.size
subjectList = subjectList.map(str) # convert to string
subjectList = subjectList.values.tolist() # convert to python list
subjectAlias = np.linspace(1,numSubjects,numSubjects).tolist()
trialNames = ['NN','YN','NY','YY'] # NN = NNL + NNH, YN = YNL + YNH, NY = NYL + NHH, YY = YYL + YYH
trialNum = [['111','112' ],['211','212'],['121','122'],['221','222']] # do not delete this line

c = 1
d = 1
trajectory = [] # variable that will store input data in [number of samples, length of seach segment, features] format
output = [] # variable that will store true output data as described above
t = 0
correspondingSubject = [-1,-1]

for subjects, subNum in zip(subjectList, subjectAlias):
    tempOutputAggregator = []
    for trials, index in zip(trialNum,range(len(trialNum))):
        tempOutputAggregator = []
        for trial in trials:
            lookFolder = cwd + '/exportedTrajectory_'+str(win)+'/'+subjects[0]+'/'+ trial # path to folder where traj.csv is stored
            tempTraj = pd.read_csv(lookFolder+'/'+ fileNom +'.csv', header=None) # read traj.csv file
            segmentSize = pd.read_csv(lookFolder+'/segmentSize.csv', header=None) # read segmentSize.csv file
            tempTraj = tempTraj.to_numpy() # convert pandas dataframe to numpy array
            segmentSize = segmentSize.to_numpy() # convert pandas dataframe to numpy array
            arrShape = tempTraj.shape # get shape of csv file1
            numSegments = arrShape[0]/segmentSize # number of trial segments in traj.csv file
            tempTrajAggregator = [] # reset tempTrajAggregator
            chunkStart = 0 # initial chunk position

             # loop to separate chunks of trajectory data and format it as [., length of each segment, features] 
            for i in range(int(numSegments[0][0])):
                # obtain a chunk of trajectory defined by segmentSize.csv file
                tempTrajChunk = tempTraj[chunkStart:chunkStart+int(segmentSize[0][0]),:] 
                arrShape = tempTrajChunk.shape # get the shape of the chunk
                # reshape the chunk to [1, length of seach segment, features]
                tempTrajChunk = tempTrajChunk.reshape(-1,arrShape[0],arrShape[1])
                chunkStart = chunkStart + int(segmentSize[0][0]) # update chunk position by segment size
                if len(tempTrajAggregator) > 0:
                    # for 1+ run, append tempTrajAggregator with data from tempTrajChunk 
                    tempTrajAggregator = np.append(tempTrajAggregator,tempTrajChunk, axis=0)
                if len(tempTrajAggregator) == 0:
                    # for 1st run, set tempTrajAggregator equal to tempTrajChunk
                    tempTrajAggregator = tempTrajChunk

                if len(correspondingSubject) > 0:
                    # for 1+ run, append tempTrajAggregator with data from tempTrajChunk 
                    correspondingSubject.append(subNum)
                if len(correspondingSubject) == 0:
                    # for 1st run, set correspondingSubject equal to subNum
                    correspondingSubject = subNum

            for i in range(int(numSegments[0][0])):
                tempOutput = 0*np.ones((1,len(trialNum)))
                tempOutput[0][index] = 1
                if len(tempOutputAggregator) > 0:
                    # for 1+ run, append tempOutputAggregator with data from tempOutput 
                    tempOutputAggregator = np.append(tempOutputAggregator,tempOutput, axis=0)
                if len(tempOutputAggregator) == 0:
                    # for 1st run, set tempOutputAggregator equal to tempOutput
                    tempOutputAggregator = tempOutput
            
            if c == 1:
                # for 1st run, set trajectory equal to tempTrajAggregator
                trajectory = tempTrajAggregator
            if c > 1:
                # for 1+ run, append trajectory with data from tempTrajAggregator 
                trajectory = np.append(trajectory,tempTrajAggregator, axis=0)
            c = c + 1
        
        if d == 1:
            # for 1st run, set output equal to tempOutputAggregator
            output = tempOutputAggregator
        if d > 1:
            # for 1+ run, append output with data from tempOutputAggregator 
            output = np.append(output,tempOutputAggregator, axis = 0)
        d = d + 1
correspondingSubject = correspondingSubject[2: len(correspondingSubject)]
arrShape = trajectory.shape
seq_length = arrShape[1] # length of each segment
numSamples = arrShape[0] # number of samples
features = arrShape[2] # data dimensionality

trajectoryDataFrame = []
for i in range(trajectory.shape[1]):
    #writematrix([xDot',yDot',zDot',subjectTurnRate',subjectHeading,subjectGroundSpeed', subjectSpeed', subjectAccel', subjectZPos, subjectHeightDot'],saveTrajFile);
    df = pd.DataFrame(trajectory[:,i,:], columns = ['xDot', 'yDot', 'zDot','thetaDot','theta', 'groundSpeed', 'speed', 'accel', 'height', 'heightRate','cumulativeSum'], index = correspondingSubject)
    trajectoryDataFrame.append(df)
outputDataFrame =  pd.DataFrame(output, columns = ['NN', 'YN', 'NY', 'YY'], index = correspondingSubject)

In [10]:
trajectory.shape

(4367, 240, 11)

In [5]:
import pickle
# Open a file and use dump() 
with open('trajectoryDataFrame_win10.pkl', 'wb') as file: 
    pickle.dump(trajectoryDataFrame, file)
with open('outputDataFrame_win10.pkl', 'wb') as file: 
    pickle.dump(outputDataFrame, file)